# descriptor

## simple descriptor

In [ ]:
class Quantity:

    def __init__(self, storage_name):
        self.storage_name = storage_name

    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('value must be > 0')

class LineItem3:
    weight = Quantity('weight')
    price = Quantity('price')

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

raisins = LineItem3('Golden raisins', 10, 6.95)
print(raisins.weight, raisins.description, raisins.price)
print(raisins.subtotal())
try:
    raisins.weight = -20
except Exception as e:
    print('set weight error')
    print(e)
finally:
    print(raisins.subtotal())

## auto save attribute

In [ ]:
class Quantity:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

class LineItem4a:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

raisins = LineItem4a('Golden raisins', 10, 6.95)
print(raisins.weight, raisins.description, raisins.price)
print(raisins.subtotal())
try:
    raisins.weight = -20
except Exception as e:
    print('set weight error')
    print(e)
finally:
    print(raisins.subtotal())

In [ ]:
class Quantity:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

class LineItem4b:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

raisins = LineItem4b('Golden raisins', 10, 6.95)
print(raisins.weight, raisins.description, raisins.price)
print(raisins.subtotal())
try:
    raisins.weight = -20
except Exception as e:
    print('set weight error')
    print(e)
finally:
    print(raisins.subtotal())

In [ ]:
class Quantity:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

class LineItem4c:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

raisins = LineItem4c('Golden raisins', 10, 6.95)
print(raisins.weight, raisins.description, raisins.price)
print(raisins.subtotal())
try:
    raisins.weight = -20
except Exception as e:
    print('set weight error')
    print(e)
finally:
    print(raisins.subtotal())

In [ ]:
def quantity():
    try:
        quantity.counter += 1
    except AttributeError:
        quantity.counter = 0

    storage_name = '_{}:{}'.format('quantity', quantity.counter)

    def qty_getter(instance):
        return getattr(instance, storage_name)

    def qty_setter(instance, value):
        if value > 0:
            setattr(instance, storage_name, value)
        else:
            raise ValueError('value must be > 0')

    return property(qty_getter, qty_setter)

class LineItem4d:
    weight = quantity()
    price = quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

raisins = LineItem4d('Golden raisins', 10, 6.95)
print(raisins.weight, raisins.description, raisins.price)
print(raisins.subtotal())
try:
    raisins.weight = -20
except Exception as e:
    print('set weight error')
    print(e)
finally:
    print(raisins.subtotal())

## complex descriptor/ validator

In [ ]:
import abc

class AutoStorage:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)

class Validated(abc.ABC, AutoStorage):

    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)

    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""

class Quantity(Validated):
    """a number greater than zero"""

    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value


class NonBlank(Validated):
    """a string with at least one non-space character"""

    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

class LineItem5a:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

raisins = LineItem5a('Golden raisins', 10, 6.95)
print(raisins.weight, raisins.description, raisins.price)
print(raisins.subtotal())
try:
    raisins.weight = -20
except Exception as e:
    print('set weight error')
    print(e)
finally:
    print(raisins.subtotal())

In [ ]:
import abc

class AutoStorage:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)

class Validated(abc.ABC, AutoStorage):

    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)

    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""

INVALID = object()

class Check(Validated):

    def __init__(self, checker):
        super().__init__()
        self.checker = checker
        if checker.__doc__ is None:
            doc = ''
        else:
            doc = checker.__doc__ + '; '
        self.message = doc + '{!r} is not valid.'

    def validate(self, instance, value):
        result = self.checker(value)
        if result is INVALID:
            raise ValueError(self.message.format(value))
        return result

def gt_zero(x):
    '''value must be > 0'''
    return x if x > 0 else model.INVALID

def non_blank(txt):
    txt = txt.strip()
    return txt if txt else model.INVALID

class LineItem5b:
    description = Check(non_blank)
    weight = Check(gt_zero)
    price = Check(gt_zero)

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

raisins = LineItem5b('Golden raisins', 10, 6.95)
print(raisins.weight, raisins.description, raisins.price)
print(raisins.subtotal())
try:
    raisins.weight = -20
except Exception as e:
    print('set weight error')
    print(e)
finally:
    print(raisins.subtotal())